In [1]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from sklearn.neighbors import LocalOutlierFactor

from DataMacpy import *

DataMacpy Setup


In [2]:
# Veriyi içeri aktaralım.

# veri yolumuzu verelim
file_path = 'train2.csv'

# verimizi okuyoruz. 
df = pd.read_csv(file_path)

print("Setup Complete")

Setup Complete


In [3]:
df.head()

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,...,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu
0,0,Kadın,31-40,Bekar,Yüksek Lisans Mezunu,Düzenli ve Ücretli Bir İşi Var,748266.440879,Büyük Şehir,Elektronik ve Teknolojik Ürünler,32010.900881,...,Etmiyor,obek_4,102.936904,1,3,2,4,3,3,4
1,1,Erkek,>60,Evli,Lise Mezunu,Kendi İşinin Sahibi,246298.609605,Küçük Şehir,Spor Malzemeleri,4145.729891,...,Etmiyor,obek_5,82.176053,3,2,2,2,1,2,3
2,2,Erkek,18-30,Bekar,Lise Mezunu,Düzenli ve Ücretli Bir İşi Var,268582.732360,Küçük Şehir,Giyim,9924.379414,...,Ediyor,obek_8,141.661426,1,2,2,2,2,3,4
3,3,Erkek,51-60,Evli,Lise Mezunu,Kendi İşinin Sahibi,327721.837865,Kırsal,Giyim,6417.780714,...,Etmiyor,obek_3,17.097397,2,2,1,3,1,2,1
4,4,Erkek,31-40,Evli,Ortaokul Mezunu,İşsiz veya Düzenli Bir İşi Yok,397431.626948,Büyük Şehir,Giyim,7886.663061,...,Etmiyor,obek_3,20.470459,1,1,2,3,1,2,1


In [4]:
# target verilerimiz string olduğu için LabelEncoder yapıyoruz. Her sınıfı bir sayıya atıyoruz.
l_fit = LabelEncoder().fit(df["Öbek İsmi"])
y = l_fit.transform(df["Öbek İsmi"])
#y = pd.DataFrame(y, columns = ["Type_1"]) # dataframe'e çevirmek için bu satırı kullanabiliriz.
y[0:20]

array([3, 4, 7, 2, 2, 7, 2, 4, 2, 5, 4, 0, 6, 2, 6, 4, 6, 5, 5, 5])

In [5]:
# labellerimizi buluyoruz.
y_label = l_fit.inverse_transform([i for i in range(max(y)+1)])
label_and_num = list(zip(y_label, [i for i in range(max(y)+1)]))
label_and_num

[('obek_1', 0),
 ('obek_2', 1),
 ('obek_3', 2),
 ('obek_4', 3),
 ('obek_5', 4),
 ('obek_6', 5),
 ('obek_7', 6),
 ('obek_8', 7)]

In [6]:
df["Öbek İsmi"] = y

In [7]:
df

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,...,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu
0,0,Kadın,31-40,Bekar,Yüksek Lisans Mezunu,Düzenli ve Ücretli Bir İşi Var,748266.440879,Büyük Şehir,Elektronik ve Teknolojik Ürünler,32010.900881,...,Etmiyor,3,102.936904,1,3,2,4,3,3,4
1,1,Erkek,>60,Evli,Lise Mezunu,Kendi İşinin Sahibi,246298.609605,Küçük Şehir,Spor Malzemeleri,4145.729891,...,Etmiyor,4,82.176053,3,2,2,2,1,2,3
2,2,Erkek,18-30,Bekar,Lise Mezunu,Düzenli ve Ücretli Bir İşi Var,268582.732360,Küçük Şehir,Giyim,9924.379414,...,Ediyor,7,141.661426,1,2,2,2,2,3,4
3,3,Erkek,51-60,Evli,Lise Mezunu,Kendi İşinin Sahibi,327721.837865,Kırsal,Giyim,6417.780714,...,Etmiyor,2,17.097397,2,2,1,3,1,2,1
4,4,Erkek,31-40,Evli,Ortaokul Mezunu,İşsiz veya Düzenli Bir İşi Yok,397431.626948,Büyük Şehir,Giyim,7886.663061,...,Etmiyor,2,20.470459,1,1,2,3,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,5455,Kadın,>60,Bekar,Üniversite Mezunu,Emekli,478054.247819,Köy veya Kasaba,Kozmetik,43182.299801,...,Etmiyor,5,51.600436,3,3,1,3,3,2,2
5456,5456,Erkek,51-60,Evli,Doktora Ötesi,Kendi İşinin Sahibi,481852.853651,Köy veya Kasaba,Ev ve Mobilya,42107.996193,...,Etmiyor,5,49.190536,2,4,1,3,3,2,2
5457,5457,Kadın,51-60,Evli,İlkokul Mezunu,Düzenli ve Ücretli Bir İşi Var,309443.687351,Küçük Şehir,Ev ve Mobilya,4655.392445,...,Etmiyor,0,9.537384,2,1,2,3,1,2,1
5458,5458,Kadın,18-30,Bekar,Yüksekokul Mezunu,Düzenli ve Ücretli Bir İşi Var,266604.268016,Büyük Şehir,Ev ve Mobilya,9016.165906,...,Ediyor,7,143.281718,1,2,2,2,2,3,4


In [8]:
object_cols, d = calc_object_cols(df, 0)
d

{'Cinsiyet': 2,
 'Yaş Grubu': 5,
 'Medeni Durum': 2,
 'Eğitim Düzeyi': 9,
 'İstihdam Durumu': 4,
 'Yaşadığı Şehir': 4,
 'En Çok İlgilendiği Ürün Grubu': 5,
 'Eğitime Devam Etme Durumu': 2}

In [9]:
# Verilerimize OrdinalEncoder uyguluyoruz. OneHot denemesi yapılacak!
ordinal_encoder = OrdinalEncoder()
df[object_cols] = ordinal_encoder.fit_transform(df[object_cols])
df

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,...,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu
0,0,1.0,1.0,0.0,5.0,0.0,748266.440879,0.0,0.0,32010.900881,...,1.0,3,102.936904,1,3,2,4,3,3,4
1,1,0.0,4.0,1.0,3.0,2.0,246298.609605,2.0,4.0,4145.729891,...,1.0,4,82.176053,3,2,2,2,1,2,3
2,2,0.0,0.0,0.0,3.0,0.0,268582.732360,2.0,2.0,9924.379414,...,0.0,7,141.661426,1,2,2,2,2,3,4
3,3,0.0,3.0,1.0,3.0,2.0,327721.837865,3.0,2.0,6417.780714,...,1.0,2,17.097397,2,2,1,3,1,2,1
4,4,0.0,1.0,1.0,4.0,3.0,397431.626948,0.0,2.0,7886.663061,...,1.0,2,20.470459,1,1,2,3,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,5455,1.0,4.0,0.0,7.0,1.0,478054.247819,1.0,3.0,43182.299801,...,1.0,5,51.600436,3,3,1,3,3,2,2
5456,5456,0.0,3.0,1.0,1.0,2.0,481852.853651,1.0,1.0,42107.996193,...,1.0,5,49.190536,2,4,1,3,3,2,2
5457,5457,1.0,3.0,1.0,8.0,0.0,309443.687351,2.0,1.0,4655.392445,...,1.0,0,9.537384,2,1,2,3,1,2,1
5458,5458,1.0,0.0,0.0,6.0,0.0,266604.268016,0.0,1.0,9016.165906,...,0.0,7,143.281718,1,2,2,2,2,3,4


In [30]:
lof = LocalOutlierFactor(novelty=True)
fit = lof.fit(df)
pred = fit.predict(df)
decision = fit.negative_outlier_factor_
decision

D:\Programlar\Anaconda3\envs\all_new\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


array([-1.04005577, -1.10650667, -1.096735  , ..., -1.13823851,
       -1.12851467, -1.06979884])

In [31]:
np.unique(pred, return_counts=True)

(array([-1,  1]), array([  61, 5399], dtype=int64))

In [32]:
# Verimizi tekrar okuyarak Encoding yapılmamış df elde ediyoruz.

# veri yolumuzu verelim
file_path2 = 'train2.csv'

# verimizi okuyoruz. 
df2 = pd.read_csv(file_path2)

print("Setup Complete")

Setup Complete


In [35]:
df2["inlier"] = pred
df2["decision"] = decision

In [37]:
rslt_df = df2[df2['inlier'] == -1]
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
rslt_df

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu,inlier,decision
155,155,Kadın,18-30,Bekar,Ortaokul Mezunu,İşsiz veya Düzenli Bir İşi Yok,1.893488e+05,Büyük Şehir,Elektronik ve Teknolojik Ürünler,9025.100309,52.240056,Ediyor,obek_8,154.036246,1,1,2,2,2,3,4,-1,-1.616348
184,184,Erkek,51-60,Evli,Üniversite Mezunu,Kendi İşinin Sahibi,4.947574e+05,Kırsal,Spor Malzemeleri,47177.288486,27.668891,Etmiyor,obek_6,54.719631,2,3,1,3,3,2,2,-1,-1.536507
204,204,Kadın,>60,Evli,Yüksek Lisans Mezunu,Emekli,4.874468e+05,Köy veya Kasaba,Ev ve Mobilya,33409.864716,26.208762,Etmiyor,obek_6,61.779529,3,3,1,3,3,2,3,-1,-1.553358
358,358,Kadın,18-30,Evli,Yüksekokul Mezunu,Düzenli ve Ücretli Bir İşi Var,1.881042e+05,Küçük Şehir,Giyim,8850.791555,53.132764,Ediyor,obek_8,108.224484,1,2,2,2,2,3,4,-1,-1.703417
383,383,Erkek,31-40,Evli,Lise Mezunu,Düzenli ve Ücretli Bir İşi Var,1.508674e+05,Büyük Şehir,Elektronik ve Teknolojik Ürünler,7772.041001,14.490522,Etmiyor,obek_3,21.305393,1,2,2,2,1,2,1,-1,-1.689374
439,439,Kadın,18-30,Bekar,Üniversite Mezunu,Düzenli ve Ücretli Bir İşi Var,1.891304e+05,Küçük Şehir,Elektronik ve Teknolojik Ürünler,9150.886913,55.171309,Ediyor,obek_8,194.795768,1,3,2,2,2,3,4,-1,-1.581495
456,456,Erkek,41-50,Bekar,Yüksek Lisans Mezunu,Emekli,3.469461e+05,Büyük Şehir,Ev ve Mobilya,19882.908904,19.550570,Etmiyor,obek_7,108.070205,2,3,2,3,2,2,4,-1,-1.791579
542,542,Kadın,31-40,Evli,Üniversite Mezunu,Düzenli ve Ücretli Bir İşi Var,3.938974e+05,Büyük Şehir,Giyim,31272.538145,19.820543,Etmiyor,obek_3,99.740468,1,3,2,3,3,2,3,-1,-2.920340
565,565,Erkek,18-30,Bekar,Üniversite Mezunu,İşsiz veya Düzenli Bir İşi Yok,1.892521e+05,Büyük Şehir,Kozmetik,10375.205347,56.694655,Ediyor,obek_8,204.553524,1,3,2,2,2,3,4,-1,-1.741577
617,617,Erkek,31-40,Bekar,Üniversite Mezunu,Düzenli ve Ücretli Bir İşi Var,3.141664e+05,Küçük Şehir,Spor Malzemeleri,29505.692703,24.387631,Ediyor,obek_7,97.474821,1,3,2,3,3,2,3,-1,-1.688698


In [22]:
df2.describe().T

,count,mean,std,min,25%,50%,75%,max
index,5460.0,2729.500000,1576.310566,0.000000,1364.750000,2729.500000,4094.250000,5.459000e+03
Yıllık Ortalama Gelir,5460.0,363571.133428,219714.442407,43922.988693,215693.439307,286925.436073,468188.221798,1.192437e+06
Yıllık Ortalama Satın Alım Miktarı,5460.0,16616.612217,14099.171704,2859.254000,4931.859057,8426.818967,30579.244695,4.860559e+04
Yıllık Ortalama Sipariş Verilen Ürün Adedi,5460.0,24.040884,14.945655,0.000000,11.550502,20.095924,35.918161,6.461620e+01
Yıllık Ortalama Sepete Atılan Ürün Adedi,5460.0,73.445693,47.214184,3.977559,25.009168,82.485579,104.473291,2.423084e+02
Yaş Durumu,5460.0,1.493773,0.673034,1.000000,1.000000,1.000000,2.000000,3.000000e+00
Eğitim Durumu,5460.0,2.281319,0.904680,1.000000,2.000000,2.000000,3.000000,4.000000e+00
Yaşadığı Şehir Durumu,5460.0,1.696154,0.459959,1.000000,1.000000,2.000000,2.000000,2.000000e+00
Yıllık Ortalama Gelir Durumu,5460.0,2.617766,0.742216,1.000000,2.000000,2.000000,3.000000,4.000000e+00
Yıllık Ortalama Satın Alım Durumu,5460.0,1.879853,0.933509,1.000000,1.000000,1.000000,3.000000,3.000000e+00


In [23]:
lof_novel = LocalOutlierFactor(novelty=True)
lof_novel.fit(df)
pred_decision = lof_novel.decision_function(df)
pred_decision

D:\Programlar\Anaconda3\envs\all_new\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


array([0.4709045 , 0.4032058 , 0.41831302, ..., 0.37139628, 0.38085928,
       0.43293681])

In [24]:
df2["pred_decision"] = pred_decision

In [26]:
rslt_df = df2[df2['inlier'] == -1]
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
rslt_df

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu,inlier,pred_decision
155,155,Kadın,18-30,Bekar,Ortaokul Mezunu,İşsiz veya Düzenli Bir İşi Yok,1.893488e+05,Büyük Şehir,Elektronik ve Teknolojik Ürünler,9025.100309,52.240056,Ediyor,obek_8,154.036246,1,1,2,2,2,3,4,-1,-0.111625
184,184,Erkek,51-60,Evli,Üniversite Mezunu,Kendi İşinin Sahibi,4.947574e+05,Kırsal,Spor Malzemeleri,47177.288486,27.668891,Etmiyor,obek_6,54.719631,2,3,1,3,3,2,2,-1,-0.027655
204,204,Kadın,>60,Evli,Yüksek Lisans Mezunu,Emekli,4.874468e+05,Köy veya Kasaba,Ev ve Mobilya,33409.864716,26.208762,Etmiyor,obek_6,61.779529,3,3,1,3,3,2,3,-1,-0.022508
311,311,Erkek,18-30,Bekar,Yüksekokul Mezunu,İşsiz veya Düzenli Bir İşi Yok,1.664032e+05,Küçük Şehir,Kozmetik,8813.629018,51.030082,Ediyor,obek_8,111.491624,1,2,2,2,2,3,4,-1,0.030010
358,358,Kadın,18-30,Evli,Yüksekokul Mezunu,Düzenli ve Ücretli Bir İşi Var,1.881042e+05,Küçük Şehir,Giyim,8850.791555,53.132764,Ediyor,obek_8,108.224484,1,2,2,2,2,3,4,-1,-0.147479
383,383,Erkek,31-40,Evli,Lise Mezunu,Düzenli ve Ücretli Bir İşi Var,1.508674e+05,Büyük Şehir,Elektronik ve Teknolojik Ürünler,7772.041001,14.490522,Etmiyor,obek_3,21.305393,1,2,2,2,1,2,1,-1,-0.143839
439,439,Kadın,18-30,Bekar,Üniversite Mezunu,Düzenli ve Ücretli Bir İşi Var,1.891304e+05,Küçük Şehir,Elektronik ve Teknolojik Ürünler,9150.886913,55.171309,Ediyor,obek_8,194.795768,1,3,2,2,2,3,4,-1,-0.024403
456,456,Erkek,41-50,Bekar,Yüksek Lisans Mezunu,Emekli,3.469461e+05,Büyük Şehir,Ev ve Mobilya,19882.908904,19.550570,Etmiyor,obek_7,108.070205,2,3,2,3,2,2,4,-1,-0.238588
542,542,Kadın,31-40,Evli,Üniversite Mezunu,Düzenli ve Ücretli Bir İşi Var,3.938974e+05,Büyük Şehir,Giyim,31272.538145,19.820543,Etmiyor,obek_3,99.740468,1,3,2,3,3,2,3,-1,-1.289013
565,565,Erkek,18-30,Bekar,Üniversite Mezunu,İşsiz veya Düzenli Bir İşi Yok,1.892521e+05,Büyük Şehir,Kozmetik,10375.205347,56.694655,Ediyor,obek_8,204.553524,1,3,2,2,2,3,4,-1,-0.162815


## Test Verisi için

In [39]:
# Veriyi içeri aktaralım.

# veri yolumuzu verelim
file_path2 = 'test_x.csv'

# verimizi okuyoruz. 
df2 = pd.read_csv(file_path2)

print("Setup Complete")

Setup Complete


In [ ]:
"""for i in df.columns[1:]:
    fig, ax = plt.subplots(figsize=(25,10))
    sns.scatterplot(data=df, x=df.index, y="Öbek İsmi", hue=i )
    plt.title(i)"""

In [43]:
def cat_summary(dataframe, col_name, plot=False, blocking=True):
    print("#"*25, col_name, "#"*25)
    print(pd.DataFrame({col_name : dataframe[col_name].value_counts(),
                       "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("\n")
    
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show(block=blocking)

In [44]:
for i in object_cols:
    cat_summary(df2, i)

######################### Cinsiyet #########################
       Cinsiyet      Ratio
Erkek      1250  53.418803
Kadın      1090  46.581197


######################### Yaş Grubu #########################
       Yaş Grubu      Ratio
18-30        710  30.341880
31-40        631  26.965812
51-60        405  17.307692
41-50        333  14.230769
>60          261  11.153846


######################### Medeni Durum #########################
       Medeni Durum      Ratio
Evli           1248  53.333333
Bekar          1092  46.666667


######################### Eğitim Düzeyi #########################
                      Eğitim Düzeyi      Ratio
Lise Mezunu                     582  24.871795
Üniversite Mezunu               506  21.623932
Yüksek Lisans Mezunu            266  11.367521
Yüksekokul Mezunu               233   9.957265
Ortaokul Mezunu                 208   8.888889
Eğitimsiz                       177   7.564103
İlkokul Mezunu                  177   7.564103
Doktora Mezunu        

In [59]:
import plotly.express as px


a = (df2.groupby(["Cinsiyet","Eğitim Düzeyi"]).agg({"index":"count"}))/df2["index"].count()
a.reset_index(inplace=True)

for i in range(0,len(df2.columns)):
    print("{} - Eğitim Düzeyi - {} - Total Ratio:".format(a.iloc[i]["Cinsiyet"],a.iloc[i]["Eğitim Düzeyi"]),a.iloc[i]["index"]*100)

fig = px.histogram(df,x="Cinsiyet", color="Eğitim Düzeyi",barmode="group",title="Cinsiyet ve Eğitim Düzeyi")
fig.show()

Erkek - Eğitim Düzeyi - Doktora Mezunu - Total Ratio: 2.3931623931623935
Erkek - Eğitim Düzeyi - Doktora Ötesi - Total Ratio: 1.9230769230769231
Erkek - Eğitim Düzeyi - Eğitimsiz - Total Ratio: 3.974358974358974
Erkek - Eğitim Düzeyi - Lise Mezunu - Total Ratio: 16.62393162393162
Erkek - Eğitim Düzeyi - Ortaokul Mezunu - Total Ratio: 3.9316239316239314
Erkek - Eğitim Düzeyi - Yüksek Lisans Mezunu - Total Ratio: 5.17094017094017
Erkek - Eğitim Düzeyi - Yüksekokul Mezunu - Total Ratio: 4.871794871794872
Erkek - Eğitim Düzeyi - Üniversite Mezunu - Total Ratio: 11.23931623931624
Erkek - Eğitim Düzeyi - İlkokul Mezunu - Total Ratio: 3.290598290598291
Kadın - Eğitim Düzeyi - Doktora Mezunu - Total Ratio: 2.1794871794871793
Kadın - Eğitim Düzeyi - Doktora Ötesi - Total Ratio: 1.6666666666666667
Kadın - Eğitim Düzeyi - Eğitimsiz - Total Ratio: 3.5897435897435894
Kadın - Eğitim Düzeyi - Lise Mezunu - Total Ratio: 8.247863247863247


In [ ]:
def loop_count(col1, col2):
    return len(col1.unique()) * len(col2.unique())

In [139]:
import plotly.express as px

def crosscheck(df, col1, col2, plot=True):
    
    
    title = f"{col1} ve {col2}"
    
    if not plot:
        print("#"*25, title, "#"*25)

    a = (df.groupby([col1, col2]).agg({"index":"count"}))/df["index"].count()
    a.reset_index(inplace=True)

    
    if plot:
        fig = px.histogram(df,x=col1, color=col2,barmode="group", title=title)
        fig.show()

    loop_no = loop_count(df[col1],df[col2])
        
    for i in range(0, loop_no):
        str1 = a.iloc[i][col1]
        str2 = a.iloc[i][col2]
        str3 = a.iloc[i]["index"]*100

        print(f"{str1} - {col2} - {str2} - Total Ratio:{str3}")

In [140]:
object_cols, d = calc_object_cols(df2, 0)
d

{'Cinsiyet': 2,
 'Yaş Grubu': 5,
 'Medeni Durum': 2,
 'Eğitim Düzeyi': 9,
 'İstihdam Durumu': 4,
 'Yaşadığı Şehir': 4,
 'En Çok İlgilendiği Ürün Grubu': 5,
 'Eğitime Devam Etme Durumu': 2}

In [141]:
look_for_columns =  object_cols.copy()
look_for_columns.remove("Eğitim Düzeyi")

for i in look_for_columns:
    crosscheck(df2, i, "Eğitim Düzeyi", plot=True)

Erkek - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:2.3931623931623935
Erkek - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:1.9230769230769231
Erkek - Eğitim Düzeyi - Eğitimsiz - Total Ratio:3.974358974358974
Erkek - Eğitim Düzeyi - Lise Mezunu - Total Ratio:16.62393162393162
Erkek - Eğitim Düzeyi - Ortaokul Mezunu - Total Ratio:3.9316239316239314
Erkek - Eğitim Düzeyi - Yüksek Lisans Mezunu - Total Ratio:5.17094017094017
Erkek - Eğitim Düzeyi - Yüksekokul Mezunu - Total Ratio:4.871794871794872
Erkek - Eğitim Düzeyi - Üniversite Mezunu - Total Ratio:11.23931623931624
Erkek - Eğitim Düzeyi - İlkokul Mezunu - Total Ratio:3.290598290598291
Kadın - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:2.1794871794871793
Kadın - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:1.6666666666666667
Kadın - Eğitim Düzeyi - Eğitimsiz - Total Ratio:3.5897435897435894
Kadın - Eğitim Düzeyi - Lise Mezunu - Total Ratio:8.247863247863247
Kadın - Eğitim Düzeyi - Ortaokul Mezunu - Total Ratio:4.957264957264957
Kadın

18-30 - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:1.2393162393162394
18-30 - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:0.6837606837606838
18-30 - Eğitim Düzeyi - Eğitimsiz - Total Ratio:1.8376068376068377
18-30 - Eğitim Düzeyi - Lise Mezunu - Total Ratio:8.333333333333332
18-30 - Eğitim Düzeyi - Ortaokul Mezunu - Total Ratio:1.5384615384615385
18-30 - Eğitim Düzeyi - Yüksek Lisans Mezunu - Total Ratio:3.5042735042735043
18-30 - Eğitim Düzeyi - Yüksekokul Mezunu - Total Ratio:5.042735042735043
18-30 - Eğitim Düzeyi - Üniversite Mezunu - Total Ratio:6.239316239316239
18-30 - Eğitim Düzeyi - İlkokul Mezunu - Total Ratio:1.9230769230769231
31-40 - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:1.3247863247863247
31-40 - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:1.2393162393162394
31-40 - Eğitim Düzeyi - Eğitimsiz - Total Ratio:1.452991452991453
31-40 - Eğitim Düzeyi - Lise Mezunu - Total Ratio:7.350427350427351
31-40 - Eğitim Düzeyi - Ortaokul Mezunu - Total Ratio:2.5213675213675213
3

Bekar - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:2.1794871794871793
Bekar - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:1.6666666666666667
Bekar - Eğitim Düzeyi - Eğitimsiz - Total Ratio:2.3931623931623935
Bekar - Eğitim Düzeyi - Lise Mezunu - Total Ratio:11.709401709401709
Bekar - Eğitim Düzeyi - Ortaokul Mezunu - Total Ratio:2.9914529914529915
Bekar - Eğitim Düzeyi - Yüksek Lisans Mezunu - Total Ratio:5.683760683760684
Bekar - Eğitim Düzeyi - Yüksekokul Mezunu - Total Ratio:6.3247863247863245
Bekar - Eğitim Düzeyi - Üniversite Mezunu - Total Ratio:11.153846153846155
Bekar - Eğitim Düzeyi - İlkokul Mezunu - Total Ratio:2.564102564102564
Evli - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:2.3931623931623935
Evli - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:1.9230769230769231
Evli - Eğitim Düzeyi - Eğitimsiz - Total Ratio:5.17094017094017
Evli - Eğitim Düzeyi - Lise Mezunu - Total Ratio:13.162393162393164
Evli - Eğitim Düzeyi - Ortaokul Mezunu - Total Ratio:5.897435897435897
Evli -

Düzenli ve Ücretli Bir İşi Var - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:2.4786324786324787
Düzenli ve Ücretli Bir İşi Var - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:1.7521367521367521
Düzenli ve Ücretli Bir İşi Var - Eğitim Düzeyi - Eğitimsiz - Total Ratio:3.034188034188034
Düzenli ve Ücretli Bir İşi Var - Eğitim Düzeyi - Lise Mezunu - Total Ratio:12.692307692307692
Düzenli ve Ücretli Bir İşi Var - Eğitim Düzeyi - Ortaokul Mezunu - Total Ratio:4.05982905982906
Düzenli ve Ücretli Bir İşi Var - Eğitim Düzeyi - Yüksek Lisans Mezunu - Total Ratio:7.264957264957266
Düzenli ve Ücretli Bir İşi Var - Eğitim Düzeyi - Yüksekokul Mezunu - Total Ratio:5.085470085470085
Düzenli ve Ücretli Bir İşi Var - Eğitim Düzeyi - Üniversite Mezunu - Total Ratio:11.367521367521368
Düzenli ve Ücretli Bir İşi Var - Eğitim Düzeyi - İlkokul Mezunu - Total Ratio:3.2478632478632483
Emekli - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:0.5555555555555556
Emekli - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:0.

Büyük Şehir - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:2.0940170940170937
Büyük Şehir - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:1.3675213675213675
Büyük Şehir - Eğitim Düzeyi - Eğitimsiz - Total Ratio:1.8376068376068377
Büyük Şehir - Eğitim Düzeyi - Lise Mezunu - Total Ratio:7.6068376068376065
Büyük Şehir - Eğitim Düzeyi - Ortaokul Mezunu - Total Ratio:2.2222222222222223
Büyük Şehir - Eğitim Düzeyi - Yüksek Lisans Mezunu - Total Ratio:5.769230769230769
Büyük Şehir - Eğitim Düzeyi - Yüksekokul Mezunu - Total Ratio:4.615384615384616
Büyük Şehir - Eğitim Düzeyi - Üniversite Mezunu - Total Ratio:9.05982905982906
Büyük Şehir - Eğitim Düzeyi - İlkokul Mezunu - Total Ratio:1.9230769230769231
Köy veya Kasaba - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:0.7692307692307693
Köy veya Kasaba - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:0.8974358974358974
Köy veya Kasaba - Eğitim Düzeyi - Eğitimsiz - Total Ratio:1.2393162393162394
Köy veya Kasaba - Eğitim Düzeyi - Lise Mezunu - Total Rati

Elektronik ve Teknolojik Ürünler - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:0.8547008547008548
Elektronik ve Teknolojik Ürünler - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:0.641025641025641
Elektronik ve Teknolojik Ürünler - Eğitim Düzeyi - Eğitimsiz - Total Ratio:0.641025641025641
Elektronik ve Teknolojik Ürünler - Eğitim Düzeyi - Lise Mezunu - Total Ratio:6.41025641025641
Elektronik ve Teknolojik Ürünler - Eğitim Düzeyi - Ortaokul Mezunu - Total Ratio:1.2393162393162394
Elektronik ve Teknolojik Ürünler - Eğitim Düzeyi - Yüksek Lisans Mezunu - Total Ratio:1.4957264957264957
Elektronik ve Teknolojik Ürünler - Eğitim Düzeyi - Yüksekokul Mezunu - Total Ratio:1.8376068376068377
Elektronik ve Teknolojik Ürünler - Eğitim Düzeyi - Üniversite Mezunu - Total Ratio:4.188034188034187
Elektronik ve Teknolojik Ürünler - Eğitim Düzeyi - İlkokul Mezunu - Total Ratio:0.8547008547008548
Ev ve Mobilya - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:1.282051282051282
Ev ve Mobilya - Eğitim Düzeyi -

Ediyor - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:0.38461538461538464
Ediyor - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:0.1282051282051282
Ediyor - Eğitim Düzeyi - Eğitimsiz - Total Ratio:0.5555555555555556
Ediyor - Eğitim Düzeyi - Lise Mezunu - Total Ratio:4.05982905982906
Ediyor - Eğitim Düzeyi - Ortaokul Mezunu - Total Ratio:0.641025641025641
Ediyor - Eğitim Düzeyi - Yüksek Lisans Mezunu - Total Ratio:1.6239316239316242
Ediyor - Eğitim Düzeyi - Yüksekokul Mezunu - Total Ratio:4.017094017094017
Ediyor - Eğitim Düzeyi - Üniversite Mezunu - Total Ratio:4.05982905982906
Ediyor - Eğitim Düzeyi - İlkokul Mezunu - Total Ratio:0.4700854700854701
Etmiyor - Eğitim Düzeyi - Doktora Mezunu - Total Ratio:4.188034188034187
Etmiyor - Eğitim Düzeyi - Doktora Ötesi - Total Ratio:3.4615384615384617
Etmiyor - Eğitim Düzeyi - Eğitimsiz - Total Ratio:7.0085470085470085
Etmiyor - Eğitim Düzeyi - Lise Mezunu - Total Ratio:20.811965811965813
Etmiyor - Eğitim Düzeyi - Ortaokul Mezunu - Total Ratio:8.

In [142]:
object_cols

['Cinsiyet',
 'Yaş Grubu',
 'Medeni Durum',
 'Eğitim Düzeyi',
 'İstihdam Durumu',
 'Yaşadığı Şehir',
 'En Çok İlgilendiği Ürün Grubu',
 'Eğitime Devam Etme Durumu']

In [144]:
look_for_columns =  object_cols.copy()
look_for_columns.remove("Yaş Grubu")

for i in look_for_columns:
    crosscheck(df2, i, "Yaş Grubu", plot=True)

Erkek - Yaş Grubu - 18-30 - Total Ratio:15.598290598290598
Erkek - Yaş Grubu - 31-40 - Total Ratio:15.512820512820513
Erkek - Yaş Grubu - 41-50 - Total Ratio:7.735042735042735
Erkek - Yaş Grubu - 51-60 - Total Ratio:7.948717948717948
Erkek - Yaş Grubu - >60 - Total Ratio:6.623931623931624
Kadın - Yaş Grubu - 18-30 - Total Ratio:14.743589743589745
Kadın - Yaş Grubu - 31-40 - Total Ratio:11.452991452991453
Kadın - Yaş Grubu - 41-50 - Total Ratio:6.495726495726497
Kadın - Yaş Grubu - 51-60 - Total Ratio:9.35897435897436
Kadın - Yaş Grubu - >60 - Total Ratio:4.52991452991453


Bekar - Yaş Grubu - 18-30 - Total Ratio:19.743589743589745
Bekar - Yaş Grubu - 31-40 - Total Ratio:14.487179487179489
Bekar - Yaş Grubu - 41-50 - Total Ratio:4.358974358974359
Bekar - Yaş Grubu - 51-60 - Total Ratio:5.213675213675214
Bekar - Yaş Grubu - >60 - Total Ratio:2.8632478632478633
Evli - Yaş Grubu - 18-30 - Total Ratio:10.598290598290598
Evli - Yaş Grubu - 31-40 - Total Ratio:12.478632478632479
Evli - Yaş Grubu - 41-50 - Total Ratio:9.871794871794872
Evli - Yaş Grubu - 51-60 - Total Ratio:12.094017094017094
Evli - Yaş Grubu - >60 - Total Ratio:8.29059829059829


Doktora Mezunu - Yaş Grubu - 18-30 - Total Ratio:1.2393162393162394
Doktora Mezunu - Yaş Grubu - 31-40 - Total Ratio:1.3247863247863247
Doktora Mezunu - Yaş Grubu - 41-50 - Total Ratio:0.9829059829059829
Doktora Mezunu - Yaş Grubu - 51-60 - Total Ratio:0.5128205128205128
Doktora Mezunu - Yaş Grubu - >60 - Total Ratio:0.5128205128205128
Doktora Ötesi - Yaş Grubu - 18-30 - Total Ratio:0.6837606837606838
Doktora Ötesi - Yaş Grubu - 31-40 - Total Ratio:1.2393162393162394
Doktora Ötesi - Yaş Grubu - 41-50 - Total Ratio:0.3418803418803419
Doktora Ötesi - Yaş Grubu - 51-60 - Total Ratio:0.5982905982905984
Doktora Ötesi - Yaş Grubu - >60 - Total Ratio:0.7264957264957265
Eğitimsiz - Yaş Grubu - 18-30 - Total Ratio:1.8376068376068377
Eğitimsiz - Yaş Grubu - 31-40 - Total Ratio:1.452991452991453
Eğitimsiz - Yaş Grubu - 41-50 - Total Ratio:1.452991452991453
Eğitimsiz - Yaş Grubu - 51-60 - Total Ratio:1.7948717948717947
Eğitimsiz - Yaş Grubu - >60 - Total Ratio:1.0256410256410255
Lise Mezunu - Yaş 

Düzenli ve Ücretli Bir İşi Var - Yaş Grubu - 18-30 - Total Ratio:17.307692307692307
Düzenli ve Ücretli Bir İşi Var - Yaş Grubu - 31-40 - Total Ratio:14.786324786324787
Düzenli ve Ücretli Bir İşi Var - Yaş Grubu - 41-50 - Total Ratio:8.461538461538462
Düzenli ve Ücretli Bir İşi Var - Yaş Grubu - 51-60 - Total Ratio:6.666666666666667
Düzenli ve Ücretli Bir İşi Var - Yaş Grubu - >60 - Total Ratio:3.7606837606837606
Emekli - Yaş Grubu - 18-30 - Total Ratio:0.3418803418803419
Emekli - Yaş Grubu - 31-40 - Total Ratio:0.8119658119658121
Emekli - Yaş Grubu - 41-50 - Total Ratio:0.8119658119658121
Emekli - Yaş Grubu - 51-60 - Total Ratio:3.888888888888889
Emekli - Yaş Grubu - >60 - Total Ratio:2.0940170940170937
Kendi İşinin Sahibi - Yaş Grubu - 18-30 - Total Ratio:4.145299145299145
Kendi İşinin Sahibi - Yaş Grubu - 31-40 - Total Ratio:4.145299145299145
Kendi İşinin Sahibi - Yaş Grubu - 41-50 - Total Ratio:1.7521367521367521
Kendi İşinin Sahibi - Yaş Grubu - 51-60 - Total Ratio:2.82051282051282

Büyük Şehir - Yaş Grubu - 18-30 - Total Ratio:14.23076923076923
Büyük Şehir - Yaş Grubu - 31-40 - Total Ratio:12.264957264957264
Büyük Şehir - Yaş Grubu - 41-50 - Total Ratio:4.572649572649572
Büyük Şehir - Yaş Grubu - 51-60 - Total Ratio:2.905982905982906
Büyük Şehir - Yaş Grubu - >60 - Total Ratio:2.5213675213675213
Köy veya Kasaba - Yaş Grubu - 18-30 - Total Ratio:3.418803418803419
Köy veya Kasaba - Yaş Grubu - 31-40 - Total Ratio:4.188034188034187
Köy veya Kasaba - Yaş Grubu - 41-50 - Total Ratio:2.1367521367521367
Köy veya Kasaba - Yaş Grubu - 51-60 - Total Ratio:6.239316239316239
Köy veya Kasaba - Yaş Grubu - >60 - Total Ratio:3.5042735042735043
Küçük Şehir - Yaş Grubu - 18-30 - Total Ratio:11.367521367521368
Küçük Şehir - Yaş Grubu - 31-40 - Total Ratio:8.717948717948717
Küçük Şehir - Yaş Grubu - 41-50 - Total Ratio:6.153846153846154
Küçük Şehir - Yaş Grubu - 51-60 - Total Ratio:4.786324786324787
Küçük Şehir - Yaş Grubu - >60 - Total Ratio:3.1623931623931623
Kırsal - Yaş Grubu -

Elektronik ve Teknolojik Ürünler - Yaş Grubu - 18-30 - Total Ratio:5.512820512820513
Elektronik ve Teknolojik Ürünler - Yaş Grubu - 31-40 - Total Ratio:5.769230769230769
Elektronik ve Teknolojik Ürünler - Yaş Grubu - 41-50 - Total Ratio:2.3504273504273505
Elektronik ve Teknolojik Ürünler - Yaş Grubu - 51-60 - Total Ratio:2.3504273504273505
Elektronik ve Teknolojik Ürünler - Yaş Grubu - >60 - Total Ratio:2.1794871794871793
Ev ve Mobilya - Yaş Grubu - 18-30 - Total Ratio:6.0256410256410255
Ev ve Mobilya - Yaş Grubu - 31-40 - Total Ratio:5.3418803418803416
Ev ve Mobilya - Yaş Grubu - 41-50 - Total Ratio:3.717948717948718
Ev ve Mobilya - Yaş Grubu - 51-60 - Total Ratio:4.572649572649572
Ev ve Mobilya - Yaş Grubu - >60 - Total Ratio:2.564102564102564
Giyim - Yaş Grubu - 18-30 - Total Ratio:8.333333333333332
Giyim - Yaş Grubu - 31-40 - Total Ratio:6.196581196581197
Giyim - Yaş Grubu - 41-50 - Total Ratio:4.401709401709402
Giyim - Yaş Grubu - 51-60 - Total Ratio:5.17094017094017
Giyim - Yaş G

Ediyor - Yaş Grubu - 18-30 - Total Ratio:12.179487179487179
Ediyor - Yaş Grubu - 31-40 - Total Ratio:1.8803418803418803
Ediyor - Yaş Grubu - 41-50 - Total Ratio:0.7264957264957265
Ediyor - Yaş Grubu - 51-60 - Total Ratio:0.7264957264957265
Ediyor - Yaş Grubu - >60 - Total Ratio:0.4273504273504274
Etmiyor - Yaş Grubu - 18-30 - Total Ratio:18.162393162393162
Etmiyor - Yaş Grubu - 31-40 - Total Ratio:25.085470085470085
Etmiyor - Yaş Grubu - 41-50 - Total Ratio:13.504273504273504
Etmiyor - Yaş Grubu - 51-60 - Total Ratio:16.58119658119658
Etmiyor - Yaş Grubu - >60 - Total Ratio:10.726495726495726
